# Prepare feature rich dataset ``data/dataset.pkl`` out of corenlp annotations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install wget

In [ ]:
import os
from pathlib import Path
import wget

W2V_MODEL_PATH ='models/'
W2V_MODEL_NAME = 'GoogleNews-vectors-negative300.bin.gz'  # 1.6G

directory = os.path.dirname(W2V_MODEL_PATH)
if not Path(directory).is_dir():
    print(f'Creating directory at {directory}',
          ' for saving word2vec pre-trained model')
    os.makedirs(directory)
if not Path(W2V_MODEL_PATH).is_file():
    w2v_archive = os.path.join(directory, W2V_MODEL_NAME)
    if not Path(w2v_archive).is_file():
        url = f'https://s3.amazonaws.com/dl4j-distribution/{W2V_MODEL_NAME}'
        print(f'Downloading word2vec pre-trained model to {w2v_archive}')
        wget.download(url, os.path.join(directory, W2V_MODEL_NAME))

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

if W2V_MODEL_NAME[-4:] in ['.vec', '.bin']:
    word2vec_model = KeyedVectors.load_word2vec_format(W2V_MODEL_PATH + W2V_MODEL_NAME,
                                                       binary=W2V_MODEL_NAME[-4:] == '.bin')
elif W2V_MODEL_NAME[-7:] == '.bin.gz':
    word2vec_model = KeyedVectors.load_word2vec_format(W2V_MODEL_PATH + W2V_MODEL_NAME, binary=True)
    
else:
    word2vec_model = Word2Vec.load(W2V_MODEL_PATH + W2V_MODEL_NAME)
    
word2vec_vector_length = len(word2vec_model.wv.get_vector('tree'))

In [ ]:
! pip install iteration_utilities

In [ ]:
_ner_kinds = ['TITLE', 'COUNTRY', 'DATE', 'PERSON', 'ORGANIZATION', 'MISC',
       'LOCATION', 'NUMBER', 'CAUSE_OF_DEATH', 'NATIONALITY', 'ORDINAL',
       'DURATION', 'CRIMINAL_CHARGE', 'CITY', 'RELIGION',
       'STATE_OR_PROVINCE', 'IDEOLOGY', 'SET', 'URL', 'PERCENT', 'TIME',
       'MONEY', 'HANDLE']

In [ ]:
_ner_kinds_obj = ['TITLE', 'COUNTRY', 'DATE', 'PERSON', 'ORGANIZATION', 'MISC',
       'LOCATION', 'NATIONALITY', 'ORDINAL',
       'DURATION', 'CITY']
    
_ner_kinds_subj = ['TITLE', 'COUNTRY', 'DATE', 'PERSON', 'ORGANIZATION', 'MISC',
   'LOCATION', 'URL']

In [ ]:
len(_ner_kinds_subj+_ner_kinds_obj)

In [ ]:
import networkx as nx
import multiprocessing
import numpy as np
from iteration_utilities import unique_everseen


def _extract_plain_features(document):
    
    def _extract(sentence):
    
        # postag sequences occuring in dataset more than 20 times
        cols_postag_subj = ['VBN~IN~NNP~IN~CD', 'VBN~IN~NNP', 'VBN~IN~CD', 'NN~NN~NN~VBN~IN~NNP', 'NNP', 'NNP~NNP~NNP~NNP', 'NN', 'JJ~NN~NN', 'JJ~CD', 'NN~CD', 'NNP~NNPS', 'NNP~NNP~NNS', 'JJ~NNP~NNP', 'RB~VBN', 'PRP', 'CD', 'NNP~NNP~IN~NNP~NNP', 'JJ~NN', 'NN~IN~NN~NN', 'CD~NNS', 'NN~IN~NNP', 'NN~NN', 'NN~NN~NN', 'JJ~NN~NN~NN', 'JJ~NN~VBN~IN~NNP', 'NN~VBN~IN~NNP', 'NNP~CD', 'NN~NN~VBN~IN~NN~NNP', 'NN~NN~VBN~IN~JJ~NN~NNP', 'NNP~NNP~NNP', 'JJ~NNS', 'NNP~NNP', 'NN~NN~CD', 'NNP~NN', 'NN~IN~CD', 'JJ~NN~NN~VBN~IN~CD', 'NN~NN~VBN~IN~CD', 'CD~NNP~CD', 'NN~NN~VBN~IN~NNP~NNP~NNP', 'NN~IN~CD~NNS', 'NNP~NNP~NN~NN', 'JJ', 'NNP~POS~NN', 'PRP$~NN~NN', 'JJ~NN~NN~NN~VBN', 'NN~NN~NN~VBN', 'NNP~NN~NN', 'JJ~NNP', 'NNP~NNP~IN~NNP', 'PRP$~JJ~NN', 'PRP$~NN', 'VBN~IN~CD~IN~NNP~NNP', 'JJ~JJ', 'PRP$~CD~NN', 'NNP~NN~VBN', 'NN~IN~NNP~NNPS', 'NN~IN~NNP~NNP', 'CD~CD~NNS', 'JJ~NN~IN~NN', 'NN~NN~NN~CD', 'RB~NN', 'VBN~IN~NNP~NNP~IN~CD', 'CD~NN', 'NN~IN~NNP~NNP~NNP', 'JJ~NN~VBN~IN~NN', 'JJ~NN~VBN', 'JJ~JJ~NN', 'JJ~JJ~NN~VBN', 'PRP$~NNS', 'CD~NNS~IN~NN', 'CD~JJ~NNS', 'NN~VBN~IN~NNP~NNP~NNP', 'JJ~NN~IN~NNP~NNP', 'NN~NNP~NNP', 'JJ~NN~NNP~NNP', 'NNS', 'NN~NN~NN~NN', 'NN~NN~IN~NN', 'NNP~IN~NNP', 'NNP~NNS', 'JJ~JJ~NN~NN', 'CD~NN~IN~NN', 'NN~CD~CD', 'NN~NN~VBN~IN~NNP', 'JJ~NN~NN~VBN~IN~NNP', 'NNP~NNP~NN', 'JJ~NN~NN~NN~CD', 'NNP~CD~NN', 'JJ~NN~NN~NN~VBN~IN~NNP', 'JJ~JJ~NN~VBN~IN~NNP', 'JJ~NN~NNS', 'NN~VBN~IN~NNP~NNP', 'JJ~NN~VBN~IN~NNP~NNP', 'RB~NNP', 'NNP~NNP~CD', 'NNS~IN~NNP', 'NN~NN~VBN', 'CD~NNS~IN~NNS', 'CD~NN~NNS', 'NN~JJ~NN', 'NNS~IN~NNP~NNP', 'NN~IN~JJ~NN', 'NNP~NNP~NNP~IN~NNP', 'NN~NNS', 'JJ~NNP~NN', 'JJ~NN~IN~NNS', 'NN~NN~IN~NNP', 'NN~IN~PRP$~NN', 'NN~IN~NN', 'JJ~VBN', 'CD~NN~NN', 'PRP$~NN~IN~NN', 'PRP$~NN~JJ~NN', 'JJ~NN~NN~VBN', 'JJ~JJ~JJ~NN', 'VBN~IN~CD~IN~NN', 'CD~TO~CD', 'NNS~NN', 'CD~NNS~VBN', 'NN~VBN', 'NNP~IN~NNP~NNP', 'NN~NNP', 'JJ~NN~NNP', 'VBN~IN~CD~TO~CD', 'JJ~NN~VBN~IN~CD', 'NN~VBN~IN~CD', 'JJ~NN~IN~NNP', 'JJ~NN~IN~JJ~NN', 'JJ~JJ~NN~NN~VBN', 'VBN~IN~NNP~NNP', 'NN~NN~IN~NNP~NNP', 'NNS~IN~NN', 'JJ~NNS~IN~NNP', 'NN~IN~NN~IN~NNP', 'NN~NN~VBN~IN~NNP~NNP', 'JJ~NN~NN~VBN~IN~NNP~NNP', 'NN~IN~JJ~NNS', 'NN~NN~CD~NN', 'NN~IN~CD~JJ~NNS', 'JJ~JJ~NNS', 'CD~JJ~NN', 'PRP$~JJ~NN~IN~NN', 'PRP$~NN~NNS', 'VBN~IN~NNP~IN~CD~TO~CD']
        cols_postag_obj = ['VBN~IN~NNP~IN~CD', 'VBN~IN~NNP', 'VBN~IN~CD', 'NN~NN~NN~VBN~IN~NNP', 'NNP', 'NNP~NNP~NNP~NNP', 'NN', 'JJ~NN~NN', 'JJ~CD', 'NN~CD', 'NNP~NNPS', 'NNP~NNP~NNS', 'JJ~NNP~NNP', 'RB~VBN', 'PRP', 'CD', 'NNP~NNP~IN~NNP~NNP', 'JJ~NN', 'NN~IN~NN~NN', 'CD~NNS', 'NN~IN~NNP', 'NN~NN', 'NN~NN~NN', 'JJ~NN~NN~NN', 'JJ~NN~VBN~IN~NNP', 'NN~VBN~IN~NNP', 'NNP~CD', 'NN~NN~VBN~IN~NN~NNP', 'NN~NN~VBN~IN~JJ~NN~NNP', 'NNP~NNP~NNP', 'JJ~NNS', 'NNP~NNP', 'NN~NN~CD', 'NNP~NN', 'NN~IN~CD', 'JJ~NN~NN~VBN~IN~CD', 'NN~NN~VBN~IN~CD', 'CD~NNP~CD', 'NN~NN~VBN~IN~NNP~NNP~NNP', 'NN~IN~CD~NNS', 'NNP~NNP~NN~NN', 'JJ', 'NNP~POS~NN', 'PRP$~NN~NN', 'JJ~NN~NN~NN~VBN', 'NN~NN~NN~VBN', 'NNP~NN~NN', 'JJ~NNP', 'NNP~NNP~IN~NNP', 'PRP$~JJ~NN', 'PRP$~NN', 'VBN~IN~CD~IN~NNP~NNP', 'JJ~JJ', 'PRP$~CD~NN', 'NNP~NN~VBN', 'NN~IN~NNP~NNPS', 'NN~IN~NNP~NNP', 'CD~CD~NNS', 'JJ~NN~IN~NN', 'NN~NN~NN~CD', 'RB~NN', 'VBN~IN~NNP~NNP~IN~CD', 'CD~NN', 'NN~IN~NNP~NNP~NNP', 'JJ~NN~VBN~IN~NN', 'JJ~NN~VBN', 'JJ~JJ~NN', 'JJ~JJ~NN~VBN', 'PRP$~NNS', 'CD~NNS~IN~NN', 'CD~JJ~NNS', 'NN~VBN~IN~NNP~NNP~NNP', 'JJ~NN~IN~NNP~NNP', 'NN~NNP~NNP', 'JJ~NN~NNP~NNP', 'NNS', 'NN~NN~NN~NN', 'NN~NN~IN~NN', 'NNP~IN~NNP', 'NNP~NNS', 'JJ~JJ~NN~NN', 'CD~NN~IN~NN', 'NN~CD~CD', 'NN~NN~VBN~IN~NNP', 'JJ~NN~NN~VBN~IN~NNP', 'NNP~NNP~NN', 'JJ~NN~NN~NN~CD', 'NNP~CD~NN', 'JJ~NN~NN~NN~VBN~IN~NNP', 'JJ~JJ~NN~VBN~IN~NNP', 'JJ~NN~NNS', 'NN~VBN~IN~NNP~NNP', 'JJ~NN~VBN~IN~NNP~NNP', 'RB~NNP', 'NNP~NNP~CD', 'NNS~IN~NNP', 'NN~NN~VBN', 'CD~NNS~IN~NNS', 'CD~NN~NNS', 'NN~JJ~NN', 'NNS~IN~NNP~NNP', 'NN~IN~JJ~NN', 'NNP~NNP~NNP~IN~NNP', 'NN~NNS', 'JJ~NNP~NN', 'JJ~NN~IN~NNS', 'NN~NN~IN~NNP', 'NN~IN~PRP$~NN', 'NN~IN~NN', 'JJ~VBN', 'CD~NN~NN', 'PRP$~NN~IN~NN', 'PRP$~NN~JJ~NN', 'JJ~NN~NN~VBN', 'JJ~JJ~JJ~NN', 'VBN~IN~CD~IN~NN', 'CD~TO~CD', 'NNS~NN', 'CD~NNS~VBN', 'NN~VBN', 'NNP~IN~NNP~NNP', 'NN~NNP', 'JJ~NN~NNP', 'VBN~IN~CD~TO~CD', 'JJ~NN~VBN~IN~CD', 'NN~VBN~IN~CD', 'JJ~NN~IN~NNP', 'JJ~NN~IN~JJ~NN', 'JJ~JJ~NN~NN~VBN', 'VBN~IN~NNP~NNP', 'NN~NN~IN~NNP~NNP', 'NNS~IN~NN', 'JJ~NNS~IN~NNP', 'NN~IN~NN~IN~NNP', 'NN~NN~VBN~IN~NNP~NNP', 'JJ~NN~NN~VBN~IN~NNP~NNP', 'NN~IN~JJ~NNS', 'NN~NN~CD~NN', 'NN~IN~CD~JJ~NNS', 'JJ~JJ~NNS', 'CD~JJ~NN', 'PRP$~JJ~NN~IN~NN', 'PRP$~NN~NNS', 'VBN~IN~NNP~IN~CD~TO~CD']
        cols_postag_rel = ['VBZ', 'VBZ~DT~JJ~NN', 'VBD~RB~VBN~IN', 'VBD~VBN~IN', 'MD~VB', 'MD~RB~VB', 'VBD~NNS~IN', 'VBD', 'NN~IN', 'VBN', 'VBN~IN', 'IN', 'VB', 'VBZ~RB~TO', 'VBZ~TO', 'RB~VBN~IN', 'VBD~RB~VBN', 'VBD~VBN', 'VBG~IN', 'VB~NN', 'VBD~RB~VBN~TO', 'VB~NN~IN', 'VBZ~VBN', 'VBD~TO', 'VBZ~IN', 'VBD~IN', 'VBZ~NN~NN', 'VBG', 'POS~NNP', 'NN~NN', 'VBZ~NN', 'VBZ~NN~IN', '', 'VBZ~VBN~IN', 'VBZ~JJ~TO', 'POS', 'VBD~NN', 'VBD~NN~IN', 'RB', 'VBP', 'VBZ~DT~NN~IN', 'VBD~JJ~NN~IN', 'VBZ~JJ~NN', 'VBZ~JJ~NN~IN', 'POS~NN', 'MD', 'CD~IN', 'VB~NNS~IN', 'NN', 'NNP~IN', 'VBD~VBN~TO', 'VBG~NN', 'VBD~NNS', 'RB~VBD', 'VBP~RB~VBN', 'VBP~VBN', 'RB~VBZ', 'NNS~IN', 'JJ~NNS~IN', 'RB~VBZ~IN', 'VBP~JJ~NNS', 'NNS', 'VBZ~RB~VBN', 'VBN~TO', 'JJ~NN', 'VBD~DT~JJ~NN', 'JJ~IN', 'RB~VBN', 'VB~NNS', 'VBZ~NN~NNS', 'VBZ~RB~VBN~IN', 'VBG~TO', 'NN~NN~NN', 'VBZ~JJ~IN', 'VBD~JJ~IN', 'VBD~JJ', 'VB~JJ', 'VBZ~VBG', 'NN~TO', 'MD~VB~IN', 'VBD~DT~NN', 'VBP~IN', 'VBZ~NNS~IN', 'RB~VBP', 'VBD~VBG', 'RB~VBG', 'JJ~NN~IN', 'JJ~JJ~NN~IN', 'VBD~JJ~TO', 'VBN~NN~IN', 'VBD~JJ~NN', 'NN~NNS', 'VBD~NN~NN', 'VBG~NNS~IN', 'VBZ~DT~NN', 'VBD~NN~TO', 'VBD~JJ~VBN~IN', 'VBP~VBN~IN', 'VBP~NNS', 'RB~VB~NNS~IN', 'VBN~NN', 'RB~VB', 'VB~IN', 'VBZ~DT~NN~TO', 'VBN~VBN', 'VBZ~JJ', 'VBZ~NN~TO', 'VBP~VBN~TO', 'TO', 'VBD~DT~NN~NN', 'VB~TO', 'VBD~NN~NNS~IN', 'VBD~DT~NN~IN', 'VBD~NNP~IN', 'VBZ~VBN~TO', 'RB~VBD~NNS~IN', 'RB~VBD~IN', 'VBG~NN~IN', 'VBD~VBN~RP~IN', 'VBD~VBN~RP', 'VBP~JJ', 'VBD~RP', 'VBD~NN~NN~IN', 'VBZ~NN~NNS~IN', 'RB~VBZ~VBN', 'VBD~NNP', 'VBP~RB~VBN~IN', 'VBP~RB', 'VBZ~NNS', 'MD~VB~RP', 'VBP~DT~NN~IN', 'VBD~DT~NN~TO', 'JJ', 'VBD~RB', 'RB~VBP~IN', 'NNP', 'RB~VBG~IN', 'VBD~NNP~NNP', 'VBG~VBN', 'VBZ~NNP~IN', 'RB~VBD~TO', 'VBZ~RB~JJ~IN', 'VBD~NNP~NNP~NNP', 'RB~VBD~NN', 'VBZ~DT~NNP', 'VBZ~JJ~JJ~NN', 'VBP~JJ~IN', 'VBZ~VBN~NN~IN', 'RBS~VBN~IN', 'VBZ~VBN~NN', 'VBZ~NNP', 'VBG~VBN~IN', 'RBS~JJ~NN~IN', 'VBD~RB~TO', 'JJ~JJ~NN', 'MD~VB~NN', 'JJ~VBN~IN', 'VB~VBN~IN', 'VBD~RP~IN', 'VBG~NNS', 'RB~VBD~NN~NN', 'VBP~NN', 'VBZ~JJS~NN', 'VBN~VBN~IN', 'VBG~DT~NN', 'IN~IN', 'RB~IN', 'VB~VBN', 'VBP~VBN~NN', 'JJ~TO', 'VBP~TO', 'VBD~JJ~NNS', 'VBD~RB~IN', 'DT~IN', 'VBD~NN~NNS', 'JJ~CC~JJ~NNS', 'JJR~NN', 'VBD~DT~JJ', 'VBD~RBS~JJ~NN', 'VBP~JJ~NNS~IN', 'VBZ~CD', 'VBZ~CD~IN', 'NN~NN~IN', 'VBD~RP~NN', 'VBP~NNS~IN', 'RB~VBD~RB', 'VBD~JJ~VBN', 'MD~NN~IN', 'RB~VBD~NN~IN', 'RB~NNP', 'VBG~RB~TO', 'VBZ~RBS~JJ~IN', 'VBG~NNP~NNS', 'VBZ~RB~JJ~NN', 'VBP~NN~IN', 'VBP~NNP', 'RB~VBG~RB', 'VBG~RB', 'VBP~RP~TO', 'VB~RB~RBR', 'VB~RBR', 'VBD~NN~NN~NNS', 'VBZ~VB~NN~IN', 'VBZ~VBG~NN~IN', 'RBR', 'NNS~NNP~IN', 'JJ~NNP', 'RB~JJ~NNP', 'NNP~NN~NN~IN', 'VBZ~NNS~RB~IN', 'VBG~CD~NN~IN', 'VBG~PRP$~NNS~TO', 'RBS~RB']

        # now we filter obj/subj/rels to be <= 3 in length
        cols_postag_subj = [seq for seq in cols_postag_subj if len(seq.split('~')) <= 3]
        cols_postag_obj = [seq for seq in cols_postag_obj if len(seq.split('~')) <= 3]
        cols_postag_rel = [seq for seq in cols_postag_rel if len(seq.split('~')) <= 3]

        def get_postags_sequence(span, words, columns):
            sequence = '~'.join([token['pos'] for token in sentence['tokens'][span[0]:span[1]] 
                         if token['originalText'] in words])

            result = tuple(int(sequence == column) for column in columns)
            return result

        def get_before(span):
            return sentence['tokens'][span[0]]['before']

        def get_after(span):
            return sentence['tokens'][span[-1] - 1]['after']

        _ner_kinds = ['TITLE', 'COUNTRY', 'DATE', 'PERSON', 'ORGANIZATION', 'MISC',
           'LOCATION', 'NUMBER', 'CAUSE_OF_DEATH', 'NATIONALITY', 'ORDINAL',
           'DURATION', 'CRIMINAL_CHARGE', 'CITY', 'RELIGION',
           'STATE_OR_PROVINCE', 'IDEOLOGY', 'SET', 'URL', 'PERCENT', 'TIME',
           'MONEY', 'HANDLE']

        _ner_kinds_obj = ['TITLE', 'COUNTRY', 'DATE', 'PERSON', 'ORGANIZATION', 'MISC',
           'LOCATION', 'NATIONALITY', 'ORDINAL',
           'DURATION', 'CITY']

        _ner_kinds_subj = ['TITLE', 'COUNTRY', 'DATE', 'PERSON', 'ORGANIZATION', 'MISC',
           'LOCATION', 'URL']

        def get_ner_occurrences(span, words, obj=True):
            mentions = [token['ner'] for token in sentence['tokens'][span[0]:span[1]] 
                                 if token['originalText'] in words]
            if obj:
                _ner_kinds = _ner_kinds_obj
            else:
                _ner_kinds = _ner_kinds_subj

            result = tuple(int(_ner_kind in mentions) for _ner_kind in _ner_kinds)
            return result

        def tag_lemma(span, words, tag=False):
            if tag:
                return [token['lemma'].lower() + '_' + _penn_tagset[token['pos']]['fPOS'] for token in sentence['tokens'][span[0]:span[1]] 
                                 if token['originalText'] in words]
            else:
                return [token['lemma'].lower() for token in sentence['tokens'][span[0]:span[1]] 
                                 if token['originalText'] in words]

        def remove_repetition(words):
            if words[:len(words)//2].strip() == words[len(words)//2:].strip():
                return words[:len(words)//2].strip()
            return words

        def _build_dep_path(dependencies, tokens, start: int, end: int):
            edges = []
            deps = {}

            for edge in dependencies:
                edges.append((edge['governor'], edge['dependent']))
                deps[(min(edge['governor'], edge['dependent']),
                      max(edge['governor'], edge['dependent']))] = edge

            graph = nx.Graph(edges)
            path = nx.shortest_path(graph, source=start, target=end)
            return path[:-1]  # exclude right end

        def _tokens_by_index(indexes, tokens):
            return [token['originalText'] for token in tokens if token['index'] in indexes]

        def _lemmas_by_index(indexes, tokens):
            return [token['lemma'].lower() for token in tokens if token['index'] in indexes]

        result = []
        header = ['subject', 'relation', 'object', 
                  'dep_path',
                  'distance_0', 'distance_1', 
                  'rel_pos_subj', 'rel_pos_rel', 'rel_pos_obj',
                  #'postags_subj', 'postags_rel', 'postags_obj',
                  'lemma_subj', 'lemma_rel', 'lemma_obj'] +\
                [ner + '_subj' for ner in _ner_kinds_subj] +\
                [ner + '_obj' for ner in _ner_kinds_obj] +\
                [col + '_subj' for col in cols_postag_subj] +\
                [col + '_obj' for col in cols_postag_obj] +\
                [col + '_rel' for col in cols_postag_rel]

        deprecated = set(['one', 'he', 'she', 'they', 'his', 'her', 'its', 'our', 'day', 'co.', 'inc.', 
              'society', 'people', 'inventor', 'head', 'poet', 'doctor', 'teacher', 'inventor', 
              'thanksgiving day', 'halloween',
              'sales person', 'model', 'board', 'technology', 'owner', 'one', 'two'])
        
        triplets = sentence['openie']
        filtered_triplets = filter(lambda obj: obj['object'].lower() not in deprecated and obj['subject'].lower() not in deprecated, triplets)
        filtered_triplets = filter(lambda obj: len(obj['object']) > 2 or len(obj['subject']) > 2, filtered_triplets)
        filtered_triplets = filter(lambda obj: len(obj['relation'].split()) < 4, filtered_triplets)
        filtered_triplets = list(filtered_triplets)
        
        for triplet in filtered_triplets:
            result.append((
                remove_repetition(triplet['subject']), 
                remove_repetition(triplet['relation']), 
                remove_repetition(triplet['object']),
                ' '.join(_lemmas_by_index(_build_dep_path(sentence['basicDependencies'], 
                                                          sentence['tokens'], 
                                                          triplet['subjectSpan'][0], 
                                                          triplet['objectSpan'][-1]), sentence['tokens'])),
                triplet['relationSpan'][0] - triplet['subjectSpan'][0],
                triplet['objectSpan'][0] - triplet['relationSpan'][0],
                triplet['subjectSpan'][0] / len(sentence['tokens']),
                triplet['relationSpan'][0] / len(sentence['tokens']),
                triplet['objectSpan'][0] / len(sentence['tokens']),
                tag_lemma(triplet['subjectSpan'], triplet['subject']),
                tag_lemma(triplet['relationSpan'], triplet['relation']),
                tag_lemma(triplet['objectSpan'], triplet['object']),
            ) +\
                get_ner_occurrences(triplet['subjectSpan'], triplet['subject'], obj=False) +\
                get_ner_occurrences(triplet['objectSpan'], triplet['object'], obj=True) +\
                get_postags_sequence(triplet['subjectSpan'], triplet['subject'], cols_postag_subj) +\
                get_postags_sequence(triplet['objectSpan'], triplet['relation'], cols_postag_obj) +\
                get_postags_sequence(triplet['relationSpan'], triplet['object'], cols_postag_rel)
            ) 

        return pd.DataFrame(result, columns=header)
    
    result = []
    for sentence in document:
        result.append(_extract(sentence))
    
    return pd.concat(result)

def _embed(placeholder, words):
    for j in range(len(words)):
        if j == len(placeholder):
            break
            
        word = words[j]
        if word and word in word2vec_model:
            placeholder[j, :] = word2vec_model[word]
    return placeholder

def _mark_ner_object(row):
    return row['relation'] + (row['DATE_obj'] == 1) * ' date'\
                           + (row['LOCATION_obj'] == 1) * ' location'
    
def _extract_features(document):
    features = _extract_plain_features(document[1])
    features.insert(loc=0, column='docid', value=document[0])
    max_len = {'obj': 3, 'rel': 3, 'subj': 3}
    features['w2v_subj'] = features['lemma_subj'].map(lambda words: _embed(np.zeros((max_len['subj'], word2vec_vector_length)), words))
    features['w2v_rel'] = features['lemma_rel'].map(lambda words: _embed(np.zeros((max_len['rel'], word2vec_vector_length)), words))
    features['w2v_obj'] = features['lemma_obj'].map(lambda words: _embed(np.zeros((max_len['obj'], word2vec_vector_length)), words))
                                                                  
    return features

def remove_repetitions(annot):
    for i in range(len(annot)):
        for j in range(len(annot[i])):
            annot[i][j]['openie'] = list(unique_everseen(annot[i][j]['openie']))
    return annot

                                           
class FeaturesProcessor:
    
    def __init__(self):
        self.pool = multiprocessing.Pool(processes=4)
                 
    def __call__(self, data):
        features = pd.concat(self.pool.map(_extract_features, data))
        return features

In [ ]:
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pandas as pd

#DATA_PATH = 'data/corenlp_annotations_ner_pairs'  #'data/filtered_annotations'
DATA_PATH = 'data/corenlp_annotations_only_ner'
RESULT_PATH = 'data/processed'
! mkdir $RESULT_PATH 
result = []
extr = FeaturesProcessor()

for file in tqdm(glob(DATA_PATH + '/*.json')):
    tmp = pd.read_json(file)
    tmp = tmp[tmp.loc[:, 1].map(len) > 0]
    tmp[1] = remove_repetitions(tmp[1].values)
    result = extr(tmp[[0, 1]].values)
    result.to_pickle(file.replace(DATA_PATH, RESULT_PATH).replace('.json', '.pkl'))

#### Collect to one file 

In [ ]:
! mv processed_ data/processed

In [ ]:
! ls -laht data/processed

In [ ]:
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pandas as pd

DATA_PATH = 'data/processed'

result = []
for file in tqdm(glob(DATA_PATH + '/*.pkl')):
    result.append(pd.read_pickle(file))
    
result = pd.concat(result)

In [ ]:
result.shape

In [ ]:
result.head()

In [ ]:
result.to_pickle('data/dataset_ner.pkl')

#### BOW for relation 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_rel = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
bow_rel = vectorizer_rel.fit_transform(result['relation'].values).toarray()
columns = [column + '_bow_rel' for column in vectorizer_rel.get_feature_names()]
features = pd.concat([result.reset_index(drop=True), pd.DataFrame(bow_rel, columns=columns)], axis=1)

In [ ]:
import pickle

pickle.dump(vectorizer_rel, open('models/relation_vectorizer.pkl', 'wb'))

#### BOW for synt dependency path

In [ ]:
vectorizer_path = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=1000)
bow_path = vectorizer_path.fit_transform(features['dep_path'].values).toarray()
columns = [column + '_bow_path' for column in vectorizer_path.get_feature_names()]
features = pd.concat([features.reset_index(drop=True), pd.DataFrame(bow_path, columns=columns)], axis=1)

In [ ]:
pickle.dump(vectorizer_path, open('models/path_vectorizer.pkl', 'wb'))

In [ ]:
features.shape

## Simple clustering example

In [1]:
import pandas as pd

features = pd.read_pickle('data/dataset_ner.pkl')

In [2]:
X = features
excluding_cols = ['docid', 'subject', 'relation', 'object', 'dep_path', 'lemma_subj', 'lemma_rel', 'lemma_obj']
embedding_cols = ['w2v_subj', 'w2v_rel', 'w2v_obj']

In [14]:
from sklearn.cluster import KMeans

kmeans=KMeans(n_clusters=100)
kmeans.fit(X.drop(columns=embedding_cols+excluding_cols))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [15]:
def show_cluster_sample(number):
    return X[kmeans.labels_ == number][['subject', 'relation', 'object']].sample(frac=1).iloc[:20].values

In [16]:
show_cluster_sample(2)

array([['Ontario Telepresence Project', 'Located at',
        'University of Toronto'],
       ['W. Ford Doolittle', 'However argued in', 'early 1980s'],
       ['New York Times', 'is In', 'late November 2005'],
       ['Ryan Cooper', 'Writing in', 'Week'],
       ['Nakamura', 'Born in', 'Japan'],
       ['ACTA', 'is In', 'early 2012 Denis'],
       ['Basilidians', 'taught According to', 'Clement'],
       ['Manning', 'serving in', 'Iraq'],
       ['James P. Hanigan', 'Writing in', 'America'],
       ['Apple', 'later announced on', 'July 17'],
       ['ENTITY', 'Founded in', '2016'],
       ['Ohmae', 'Born in', 'Kitakyūshū'],
       ['Horstman', 'living with', 'Portuguese'],
       ['first U.S. president', 'is In', '1965'],
       ['it', 'Aggregating', 'Metacritic'],
       ['Ladopoulos', 'Later worked as', 'consultant'],
       ['Tesla', 'Named after', 'engineer Nikola Tesla'],
       ['Rintaro', 'Returning to', 'past'],
       ['Pokémon Masters', 'Developed by', 'DeNA'],
       ['jou

In [17]:
with open('simple_clusters.txt', 'w') as f:
    for i in range(19):
        f.write(f'{i}-----------------\n{show_cluster_sample(i)}\n\n\n')